# Размещение баннеров

## Постановка задачи
Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к нам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Нам нужно выбрать 20 таких локаций для размещения.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.
Часть открытых данных есть, например, на сайте archive.org:
https://archive.org/details/201309_foursquare_dataset_umn


## Преобразование данных

Считаем скачанные данные и преобразуем в датафрейм
Для удобной работы с этим данными удалим строки не содержащие координат - они неинформативны для нас:

id,user_id,venue_id,latitude,longitude,created_at

984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47

984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43

984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22

...


In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift

In [2]:
df = pd.read_csv(r'D:\projects\coursera-specialization-Machine-learning-and-data-analysis\Поиск структуры в данных\checkins.dat', sep='|', skipinitialspace=True, skiprows=[1], low_memory=False, index_col=0)
df.head()

,user_id,venue_id,latitude,longitude,created_at
id,,,,,
984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


In [3]:
df.columns = [column.strip() for column in df.columns]
df.dropna(inplace=True)
df.drop(['user_id', 'venue_id', 'created_at'], axis=1, inplace=True)
df.head()

,latitude,longitude
id,,
984222,38.895112,-77.036366
984234,33.800745,-84.410520
984291,45.523452,-122.676207
984318,40.764462,-111.904565
984232,33.448377,-112.074037


In [4]:
df = df.iloc[:100000]

# Кластеризация
Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/clustering.html#mean-shift), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках, кластеризация будет работать долго (порядка часа). Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.

In [5]:
MS = MeanShift(bandwidth = 0.1)
MS.fit(df)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=None, seeds=None)

In [6]:
labels = MS.labels_
cluster_centers = MS.cluster_centers_

Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте, можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

38.8951118,-77.0363658

33.800745,-84.41052

45.5234515,-122.6762071

...

In [7]:
df['cluster'] = MS.predict(df)

In [8]:
cluster_size = pd.DataFrame(df.pivot_table(index = 'cluster', aggfunc = 'count', values = 'latitude'))
cluster_size.columns = ['clust_size']

In [9]:
cluster_centers_df = pd.DataFrame(cluster_centers)
cluster_centers_df.columns = ['cent_latitude', 'cent_longitude']

In [10]:
cluster_df = cluster_centers_df.join(cluster_size)
cluster_df.to_csv('clusters.csv', index = None)
cluster_df = cluster_df[cluster_df.clust_size > 15]
cluster_df.head()

,cent_latitude,cent_longitude,clust_size
0,40.717716,-73.991835,12506
1,33.449438,-112.002140,4692
2,33.446380,-111.901888,3994
3,41.878244,-87.629843,3363
4,37.688682,-122.409330,3526


Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

 * 33.751277, -118.188740 (_Los Angeles_)
 * 25.867736, -80.324116 (_Miami_)
 * 51.503016, -0.075479 (_London_)
 * 52.378894, 4.885084 (_Amsterdam_)
 * 39.366487, 117.036146 (_Beijing_)
 * -33.868457, 151.205134 (_Sydney_)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

In [11]:
def get_distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2) ** 0.5

office_coordinates = [
    (33.751277, -118.188740),
    (25.867736, -80.324116),
    (51.503016, -0.075479),
    (52.378894, 4.885084),
    (39.366487, 117.036146),
    (-33.868457, 151.205134)
]

def get_min_distance_to_office(lat, lon):
    min_dist = None
    for (of_lat, of_lon) in office_coordinates:
        dist = get_distance(lat, lon, of_lat, of_lon)
        if (min_dist is None) or (dist < min_dist):
            min_dist = dist
    return min_dist

In [12]:
cluster_df.head()
cluster_df['min_distance'] = np.vectorize(get_min_distance_to_office)(cluster_df.cent_latitude, cluster_df.cent_longitude)

In [13]:
cluster_df.sort_values('min_distance')[:20]

,cent_latitude,cent_longitude,clust_size,min_distance
420,-33.860630,151.204776,28,0.007835
370,52.372964,4.892317,31,0.009353
419,25.845672,-80.318891,28,0.022674
58,51.502991,-0.125537,254,0.050058
51,33.809878,-118.148924,281,0.070848
29,25.785812,-80.217938,564,0.134109
167,25.705350,-80.283429,80,0.167406
92,26.010098,-80.199991,138,0.188876
87,33.888325,-118.048928,100,0.195779
42,33.872986,-118.362091,384,0.211811
